## Qwen3-14B's Fine-Tuning Test

official
> ref: https://docs.unsloth.ai/models/qwen3-how-to-run-and-fine-tune#fine-tuning-qwen3-with-unsloth
> ref: https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Qwen3_(14B)-Reasoning-Conversational.ipynb

note
> ref: https://note.com/sachi2222/n/na7b1d91ffb5d

In [2]:
# imports
## LLM
import torch
from transformers import TrainingArguments

from unsloth import FastLanguageModel, is_bfloat16_supported
from trl import SFTTrainer, SFTConfig


## Dataset
from datasets import load_dataset, Dataset
from unsloth import to_sharegpt
import re

## Load & Setting Model

In [4]:
# model loading
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen3-14B",
    max_seq_length=40960, # context length
    load_in_4bit=True, # less mem mode
    load_in_8bit=False, # more mem mode
    full_finetuning=False # if u need full-FT
)

==((====))==  Unsloth 2025.9.11: Fast Qwen3 patching. Transformers: 4.56.2.
   \\   /|    NVIDIA RTX A6000. Num GPUs = 1. Max memory: 47.431 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.6. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


<string>:45: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
# select LoRA option
model = FastLanguageModel.get_peft_model(
    model,
    r=32, # Choose any num like 8, 16, 32, 64, 128
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
    lora_alpha=32, # alpha = r | r*2
    lora_dropout=0, # 0 is optimized
    bias="none", # "none" is optimized
    use_gradient_checkpointing="unsloth", # True | "unsloth", for too-long context
    random_state=3407,
    use_rslora=False, # under here, optical
    loftq_config=None
)

Unsloth 2025.9.11 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


## Convert Datasets

In [46]:
# make Datasets
# load json
raw_data = load_dataset("json", data_files="./jvn_results_wordpress202304_conv_no-id.json")

# convert hugginface-style
huggingface_data = Dataset.from_list(raw_data['train'])

# convert share_gpt-style
# Three dialogue sessions are preferred
share_data = to_sharegpt(huggingface_data,
                        merged_prompt="{instruction}",
                        merged_column_name="instruction",
                        output_column_name="output",
                        conversation_extension=3)


Merging columns:   0%|          | 0/9742 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/9742 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/9742 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/9742 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/9742 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/9742 [00:00<?, ? examples/s]

In [47]:
# Minor format changes
converted_share_data = [[{
            "role": "user" if message["from"] == "human" else "assistant",
            "content": re.sub(r"\('(.+?)',\)", r"\1", message["value"])
        }
        for message in item["conversations"]
    ]
    for item in share_data
]

In [45]:
converted_share_data[0]

[{'role': 'user', 'content': 'JVNDB-2025-010225 について教えてください'},
 {'role': 'assistant',
  'content': 'JVNDB-2025-010225 とは eMagicOne の WordPress 用 eMagicOne Store Manager for WooCommerce におけるファイル名やパス名の外部制御に関する脆弱性 のことです。The eMagicOne Store Manager for WooCommerce plugin for WordPress is vulnerable to arbitrary file deletion due to insufficient file path validation in the delete_file() function in all versions up to, and including, 1.2.5. This makes it possible for unauthenticated attackers to delete arbitrary files on the server, which can easily lead to remote code execution when the right file is deleted (such as wp-config.php). This is only exploitable by unauthenticated attackers in default configurations where the the default password is left as 1:1, or where the attacker gains access to the credentials. この脆弱性を受けるバージョンは eMagicOne\neMagicOne Store Manager for WooCommerce 1.2.5 およびそれ以前 です'},
 {'role': 'user', 'content': 'JVNDB-2023-017528 について教えてください'},
 {'role': 'assistant',
  'conten

In [48]:
# Addition of <\llm_start\>, etc
conversations = tokenizer.apply_chat_template(
    converted_share_data,
    tokenize = False,
)

In [49]:
# Converted to Hugging Face format

targetdataset = Dataset.from_dict({"text": conversations})

## Train settings

In [52]:
# Train
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=targetdataset,
    eval_dataset=None, # test Datasets
    args=SFTConfig(
        dataset_text_field="text",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        # num_train_epochs=1,
        max_steps=30,
        learning_rate=2e-4,
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        report_to="none"
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=24):   0%|          | 0/9742 [00:00<?, ? examples/s]

## Start FT

In [ ]:
trainer_stats = trainer.train()

## Inference

In [53]:
# Inference test
prompt="""
hello! What is your special skill?
"""

messages = [{"role" : "user", "content" : prompt}]

text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = True, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 4096, # Increase for longer outputs!
    temperature = 0.6,
    top_p = 0.95,
    top_k = 20,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)


In [53]:
# Inference test2
prompt="""
You are a white hat hacker tasked with discovering vulnerabilities in the provided source code.
Perform the following three actions on the source code below:
1. Identify the vulnerability
2. Present the risks of leaving it unaddressed
3. Provide a solution to eliminate the vulnerability

Ensure your output adheres to the following three points:
1. Output in Japanese
2. Be clear and concise
3. Use Markdown format

The source code is shown below:
---

"""

messages = [{"role" : "user", "content" : prompt}]

text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
    enable_thinking = True, # Disable thinking
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 4096, # Increase for longer outputs!
    temperature = 0.6,
    top_p = 0.95,
    top_k = 20,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)


## Save LoRA/Model

In [ ]:
# save LoRA
model.save_pretrained("Vulnerability_Detection_Wordpress")
tokenizer.save_pretrained("Vulnerability_Detection_Wordpress")

In [ ]:
# save ALL-MODEL
model.save_pretrained_merged("Vulnerability_Detection_Wordpress", tokenizer, save_method="merged_16bit")

In [ ]:
# save ALL-MODEL as .gguf
model.save_pretrained_merged("Vulnerability_Detection_Wordpress", tokenizer)

## Upload LoRA/Model

In [ ]:
# upload
